# Links

https://python.langchain.com/docs/how_to/custom_tools/

https://python.langchain.com/docs/tutorials/agents/

Tool collection:

https://python.langchain.com/docs/integrations/tools/

How to pass tool outputs back to LLMs:

https://python.langchain.com/docs/how_to/tool_results_pass_to_model/

Youtube tool:

https://medium.com/mcd-unison/youtube-data-api-v3-in-python-tutorial-with-examples-e829a25d2ebd


In [5]:
%run utilities.ipynb
llm_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0) 

#llm_model = ChatOpenAI(model="gpt-4o", temperature=0) 


# LLM Ohne Tool

In [13]:
query=  f""""whats 5 times forty plus two plus 2^4 and the whole thing squared.
        Take this result for the upper limit of an intergal over log(x) and lower limit=1, use latex formulas to write calculations."""
#Answer: 464262.48672797733220295225282993696497242417744988865603112297516
#        Then take the square root and say good-bye"""
#Answer=681.36
answer=llm_model.invoke(query)

In [14]:
print(answer.content)

To calculate the expression:

\[ (5 \times 40 + 2 + 2^4)^2 = (200 + 2 + 16)^2 = 218^2 = 47524 \]

Now, we can set up the integral using the upper limit as 47524 and the lower limit as 1:

\[ \int_{1}^{47524} \log(x) \, dx \]


# LLM mit Wolfram Mathe Tool und react agent

Zuerst eine WOlfram API erstellen: https://developer.wolframalpha.com/access

Hier ist eine kurze Erklärung des Codes:

1. **Tool laden**: `load_tools(["wolfram-alpha"])` lädt das Wolfram Alpha-Tool und speichert es in `wolfram_tool`.
2. **Tools initialisieren**: Die geladenen Tools werden in eine Liste `tools` eingefügt, um sie später an den Agenten zu binden.
3. **Agenten erstellen**: `create_react_agent(llm_model, tools)` erstellt einen `agent_executor`, der das Sprachmodell (`llm_model`) mit den Tools kombiniert und auf Anfragen reagieren kann.
4. **Anfrage verarbeiten**: `agent_executor.invoke({"messages": [HumanMessage(content=query)]})` führt die Anfrage (`query`) aus und gibt die Antwort des Agenten zurück.

Dieser Aufbau ermöglicht es dem Agenten, auf Anfragen zu reagieren und das Wolfram Alpha-Tool dynamisch zur Berechnung oder Informationsabfrage zu verwenden.

In [15]:
from langchain.agents import load_tools
wolfram_tool = load_tools(["wolfram-alpha"])
tools = [*wolfram_tool]
from langgraph.prebuilt import create_react_agent
agent_executor = create_react_agent(llm_model, tools)
response = agent_executor.invoke({"messages": [HumanMessage(content=query)]})

In [16]:
for i in response["messages"]:
    print(i.name)  
    print(i.content)
    

None
"whats 5 times forty plus two plus 2^4 and the whole thing squared.
        Take this result for the upper limit of an intergal over log(x) and lower limit=1, use latex formulas to write calculations.
None

wolfram_alpha
Assumption: (5×40 + 2 + 2^4)^2 
Answer: 47524
None

wolfram_alpha
Assumption: integral_1^47524 log(x) dx = 47524 log(47524) - 47523≈464260. 
Answer: integral_1^47524 log(x) dx = 47524 log(47524) - 47523≈464260.
None
The calculation is as follows:

Let \( y = (5 \times 40 + 2 + 2^4)^2 = 47524 \)

The integral over \( \log(x) \) from 1 to \( y \) is:

\[ \int_{1}^{47524} \log(x) \, dx = 47524 \log(47524) - 47523 \approx 464260 \]


# Tools

Dekoratoren (@tool):
Die Funktionen sind mit dem Dekorator @tool markiert. Dies deutet darauf hin, dass diese Funktionen  Teil eines größeren Systems sind, in dem Tools für bestimmte Zwecke verwendet werden (z. B. in einem Chatbot-Framework oder einer API). Der Dekorator @tool wird  verwendet, um diese Funktionen zu registrieren, damit sie programmatisch oder dynamisch aufgerufen werden können.

Funktionen:
multiply(a: int, b: int) -> int:

Diese Funktion nimmt zwei ganze Zahlen (a und b) als Eingabe und gibt deren Produkt zurück.
Die Typangaben (-> int) zeigen an, dass die Ausgabe ebenfalls eine ganze Zahl ist.
Dies ist eine einfache Hilfsfunktion zum Multiplizieren zweier Zahlen.
add(first_int: int, second_int: int) -> int:

Diese Funktion addiert zwei Ganzzahlen, first_int und second_int, und gibt das Ergebnis zurück.
Der Zweck besteht darin, eine einfache Addition durchzuführen.
exponentiate(base: int, exponent: int) -> int:


In [17]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int



# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


Der Einsatz von async kann in größeren Anwendungen nützlich sein, bei denen jede Funktion (auch einfache wie die Multiplikation) in eine asynchrone Eventschleife passen muss, was der Fall sein könnte, 
wenn diese Funktion in einem Webframework oder Chatbot-System mit asynchroner Anfragebearbeitung verwendet wird.

In [18]:
# Asynch
@tool
async def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

## Exercise: 

Erzeuge ein tool (exponentiate), welches Zahlen exponentiert. Verwende multiply, add und exponentiate  mit dem react agent, um eine Matheaufgabe zu lösen

In [12]:

tools = [multiply, add, exponentiate]

NameError: name 'exponentiate' is not defined